In [1]:
import sounddevice as sd
import numpy as np
import scipy.io.wavfile as wav

### Getting Audio

In [ ]:
duration = 5
samplerate = 44100
print(sd.query_devices())

In [3]:
# Query devices to make sure the chosen one is available and its parameters
print(sd.query_devices(84))  # Change the index and hostapi as needed

# Define the duration, sample rate, and device ID
duration = 10  # seconds
samplerate = 48000  # Hertz

# Record audio from the specified device
print("Recording from device ID:", 84)
myrecording = sd.rec(frames=int(duration * samplerate),
                     samplerate=samplerate,
                     channels=2,
                     dtype='float32',
                     device=84)
sd.wait()  # Wait until recording is finished
print("Done recording.")

{'name': 'Wave Link Stream (Wave Link Stream)', 'index': 84, 'hostapi': 4, 'max_input_channels': 2, 'max_output_channels': 0, 'default_low_input_latency': 0.01, 'default_low_output_latency': 0.01, 'default_high_input_latency': 0.08533333333333333, 'default_high_output_latency': 0.08533333333333333, 'default_samplerate': 48000.0}
Recording from device ID: 84
Done recording.


In [4]:
myrecording_int16 = (np.iinfo(np.int16).max * myrecording).astype(np.int16)

wav.write('output.wav', samplerate, myrecording_int16)

### Sedning  audio file to openai api for transcription

In [1]:
import json

def get_keys(path):
    with open(path) as f:
        return json.load(f)

key = get_keys('../../../../.secret/secrets.json')["OPENAI_API_KEY"]

In [6]:
from openai import OpenAI
client = OpenAI(api_key=key)

audio_file= open("output.wav", "rb")
transcript = client.audio.transcriptions.create(
  model="whisper-1", 
  file=audio_file
)

In [7]:
transcript.text

'When using the inet ntop function, how do I get the port the client connected on?'

### Sending audio transcript to chatgpt

In [7]:
response = client.chat.completions.create(
    model="gpt-3.5-turbo-1106",
    messages=[
        {
          "role": "system",
          "content": "You are a helpful assistant that is helping a student learn the C programming language. You should never respond with code but rather explanations so the student can better understand the problem and answer at hand."
        },
        {
          "role": "user",
          "content": transcript.text,
        },    
    ],
    temperature=1,
    max_tokens=256,
    top_p=1,
    frequency_penalty=0,
    presence_penalty=0
)

In [8]:
response.choices[0].message.content

"When using the `inet_ntop` function, you can't directly get the port the client connected on because `inet_ntop` function only converts the network address from binary to presentation format. To get the port the client connected on, you need to use the `struct sockaddr_in` or `struct sockaddr_in6` (depending on whether you are using IPv4 or IPv6) to store client address information.\n\nAfter accepting a connection using the `accept` function, you can use the `getpeername` function to retrieve the address and port information into a `struct sockaddr` and then cast it to `struct sockaddr_in` or `struct sockaddr_in6` depending on the protocol.\n\nOnce you have the `struct sockaddr_in` or `struct sockaddr_in6` you can then access the port using the `sin_port` member for IPv4, or the `sin6_port` member for IPv6.\n\nRemember that network byte order (big-endian) is used for port numbers, so you may need to convert the port number to host byte order (using the `ntohs` function) before using i

### Sending chatgpt to whisper for voice

In [9]:
response = client.audio.speech.create(
    model="tts-1",
    voice="shimmer",
    input=response.choices[0].message.content,
)

response.stream_to_file("answer.mp3")